In [1]:
from langchain import hub
from langchain.agents import create_structured_chat_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.tools import BaseTool, Tool
from langchain_experimental.agents.agent_toolkits import create_csv_agent, create_python_agent
from langchain_experimental.tools import PythonREPLTool
from langchain_openai import ChatOpenAI

In [2]:
model = ChatOpenAI(model='gpt-3.5-turbo')

In [3]:
class TextLengthTool(BaseTool):
    name = "文本字数计算工具"
    description = "当你需要计算文本包含的字数时，使用此工具"

    def _run(self, text):
        return len(text)

In [4]:
python_agent_executor = create_python_agent(
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    tool=PythonREPLTool(),
    verbose=True,
    agent_executor_kwargs={"handle_parsing_errors": True}
)

In [5]:
csv_agent_executor = create_csv_agent(
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0,),
    path="house_price.csv",
    verbose=True,
    agent_executor_kwargs={"handle_parsing_errors": True}
)

In [6]:
tools=[
    Tool(
        name="Python代码工具",
        description="""当你需要借助Python解释器时，使用这个工具。
        用自然语言把要求给这个工具，它会生成Python代码并返回代码执行的结果。""",
        func=python_agent_executor.invoke
    ),
    Tool(
        name="CSV分析工具",
        description="""当你需要回答有关house_price.csv文件的问题时，使用这个工具。
        它接受完整的问题作为输入，在使用Pandas库计算后，返回答案。""",
        func=csv_agent_executor.invoke
    ),
    TextLengthTool()
]

In [7]:
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

In [8]:
prompt = hub.pull("hwchase17/structured-chat-agent")
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'structured-chat-agent', 'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action 

In [9]:
agent = create_structured_chat_agent(
    llm=model,
    tools=tools,
    prompt=prompt
)

In [10]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, 
    tools=tools, 
    memory=memory, 
    verbose=True, 
    handle_parsing_errors=True
)

In [11]:
agent_executor.invoke({"input": "第8个斐波那契数列的数字是多少？"})



> Entering new AgentExecutor chain...
{
  "action": "Python代码工具",
  "action_input": "fib = lambda n: n if n <= 1 else fib(n-1) + fib(n-2); fib(8)"
}

> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


We are given a lambda function that calculates the Fibonacci sequence recursively. We need to find the 8th Fibonacci number.
Action: Python_REPL
Action Input: fib = lambda n: n if n <= 1 else fib(n-1) + fib(n-2); fib(8)
Observation: 
Thought:The code is executing the lambda function to find the 8th Fibonacci number.
Final Answer: 21

> Finished chain.
{'input': 'fib = lambda n: n if n <= 1 else fib(n-1) + fib(n-2); fib(8)', 'output': '21'}{
  "action": "Final Answer",
  "action_input": 21
}

> Finished chain.


{'input': '第8个斐波那契数列的数字是多少？',
 'chat_history': [HumanMessage(content='第8个斐波那契数列的数字是多少？'),
  AIMessage(content='21')],
 'output': 21}

In [12]:
agent_executor.invoke({"input": "house_price数据集里，所有房子的价格平均值是多少？用中文回答"})



> Entering new AgentExecutor chain...
{
  "action": "CSV分析工具",
  "action_input": "What is the average price of all houses in the dataset?"
}

> Entering new AgentExecutor chain...
Thought: To find the average price, we need to calculate the mean of the 'price' column in the dataframe.
Action: python_repl_ast
Action Input: df['price'].mean()4766729.247706422I now know the final answer
Final Answer: The average price of all houses in the dataset is $4,766,729.25.

> Finished chain.
{'input': 'What is the average price of all houses in the dataset?', 'output': 'The average price of all houses in the dataset is $4,766,729.25.'}{
  "action": "Final Answer",
  "action_input": "数据集中所有房屋的平均价格是4766729.25"
}

> Finished chain.


{'input': 'house_price数据集里，所有房子的价格平均值是多少？用中文回答',
 'chat_history': [HumanMessage(content='第8个斐波那契数列的数字是多少？'),
  AIMessage(content='21'),
  HumanMessage(content='house_price数据集里，所有房子的价格平均值是多少？用中文回答'),
  AIMessage(content='数据集中所有房屋的平均价格是4766729.25')],
 'output': '数据集中所有房屋的平均价格是4766729.25'}

In [13]:
agent_executor.invoke({"input": "'君不见黄河之水天上来奔流到海不复回'，这句话的字数是多少？"})



> Entering new AgentExecutor chain...
{
  "action": "文本字数计算工具",
  "action_input": {
    "text": "君不见黄河之水天上来奔流到海不复回"
  }
}17{
  "action": "Final Answer",
  "action_input": "17"
}

> Finished chain.


{'input': "'君不见黄河之水天上来奔流到海不复回'，这句话的字数是多少？",
 'chat_history': [HumanMessage(content='第8个斐波那契数列的数字是多少？'),
  AIMessage(content='21'),
  HumanMessage(content='house_price数据集里，所有房子的价格平均值是多少？用中文回答'),
  AIMessage(content='数据集中所有房屋的平均价格是4766729.25'),
  HumanMessage(content="'君不见黄河之水天上来奔流到海不复回'，这句话的字数是多少？"),
  AIMessage(content='17')],
 'output': '17'}